In [49]:
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
from matplotlib import pyplot

%matplotlib inline

In [50]:
adidas_path = 'Resources/Adidas US Sales Datasets.xlsx'
adidas_df = pd.read_excel(adidas_path,
                          index_col='Unnamed: 3',
                          parse_dates=True,
                          date_format='dateutil')

In [51]:
adidas_df = adidas_df.rename_axis('Invoice Date')
adidas_df.drop(['Unnamed: 0', 'Unnamed: 2'], axis=1, inplace=True)
adidas_df = adidas_df.rename(
    columns={
        'Unnamed: 1': 'Retailer',
        'Unnamed: 4': 'Region',
        'Unnamed: 5': 'State',
        'Unnamed: 6': 'City',
        'Unnamed: 7': 'Product',
        'Unnamed: 8': 'Price per Unit',
        'Unnamed: 9': 'Units Sold',
        'Unnamed: 10': 'Total Sales',
        'Unnamed: 11': 'Operating Profit',
        'Unnamed: 12': 'Operating Margin',
        'Unnamed: 13': 'Sales Method'
    }
)
adidas_df.dropna(inplace=True)

In [52]:
display(adidas_df.head(20))
display(adidas_df.tail(20))

,Retailer,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
Invoice Date,,,,,,,,,,,
Invoice Date,Retailer,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
2020-01-01 00:00:00,Foot Locker,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000,0.5,In-store
2020-01-02 00:00:00,Foot Locker,Northeast,New York,New York,Men's Athletic Footwear,50,1000,500000,150000,0.3,In-store
2020-01-03 00:00:00,Foot Locker,Northeast,New York,New York,Women's Street Footwear,40,1000,400000,140000,0.35,In-store
2020-01-04 00:00:00,Foot Locker,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875,0.35,In-store
2020-01-05 00:00:00,Foot Locker,Northeast,New York,New York,Men's Apparel,60,900,540000,162000,0.3,In-store
2020-01-06 00:00:00,Foot Locker,Northeast,New York,New York,Women's Apparel,50,1000,500000,125000,0.25,In-store
2020-01-07 00:00:00,Foot Locker,Northeast,New York,New York,Men's Street Footwear,50,1250,625000,312500,0.5,In-store
2020-01-08 00:00:00,Foot Locker,Northeast,New York,New York,Men's Athletic Footwear,50,900,450000,135000,0.3,Outlet


,Retailer,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
Invoice Date,,,,,,,,,,,
2021-11-18 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42,114,4788,2058.84,0.43,Outlet
2021-11-18 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Women's Street Footwear,51,114,5814,2034.9,0.35,Outlet
2021-11-18 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Women's Athletic Footwear,43,114,4902,1666.68,0.34,Outlet
2021-11-18 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Men's Apparel,59,96,5664,1925.76,0.34,Outlet
2021-11-18 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Women's Apparel,61,120,7320,3001.2,0.41,Outlet
2021-12-17 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Men's Street Footwear,49,200,9800,4606,0.47,Outlet
2021-12-17 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,49,128,6272,2634.24,0.42,Outlet
2021-12-17 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Women's Street Footwear,47,128,6016,2165.76,0.36,Outlet
2021-12-17 00:00:00,Foot Locker,Northeast,New Hampshire,Manchester,Women's Athletic Footwear,49,116,5684,1989.4,0.35,Outlet


In [53]:
monthly_temps_path = 'Resources/average_monthly_temperature_by_state_1950-2022.csv'
monthly_temps_df = pd.read_csv(monthly_temps_path,
                               index_col='year')

In [54]:
monthly_temps_df = monthly_temps_df.loc['2020':'2021']

monthly_temps_df.drop(['Unnamed: 0', 'monthly_mean_from_1901_to_2000', 'centroid_lon', 'centroid_lat'], axis=1, inplace=True)

monthly_temps_df = monthly_temps_df

In [57]:

display(monthly_temps_df.head(48))
display(monthly_temps_df.tail(48))

,month,state,average_temp
year,,,
2020,1,Alabama,50.4
2020,1,Arizona,43.0
2020,1,Arkansas,44.8
2020,1,California,44.8
2020,1,Colorado,27.8
2020,1,Connecticut,32.4
2020,1,Delaware,40.1
2020,1,Florida,62.8
2020,1,Georgia,51.8


,month,state,average_temp
year,,,
2021,12,Alabama,54.6
2021,12,Arizona,50.7
2021,12,Arkansas,51.4
2021,12,California,49.2
2021,12,Colorado,36.5
2021,12,Connecticut,39.0
2021,12,Delaware,44.6
2021,12,Florida,64.9
2021,12,Georgia,54.6
